In [48]:
import numpy as np
from numpy import linalg as LA

In [41]:
D = 1000
U = np.zeros((D, 9))
V = np.zeros((D, 9))

In [42]:
for i in range(0, 9):
    U[100*i:100*(i+2),i] = (np.random.rand(200,1)/10).reshape(200,);
    V[100*i:100*(i+2),i] = (np.random.rand(200,1)/10).reshape(200,);

In [ ]:
A = np.matmul(U,V.transpose())
eigenvalues = LA.eig(A)
max_eigenvalue = max(eigenvalues[0])
A = 0.8*A/max_eigenvalue

In [43]:
mu = np.random.rand(D,1)/D
w = 1

In [46]:
def generate_samples(w, mu, A, num_sequences, max_events_per_sequence):
    pass

In [47]:
num_sequences = 250
max_events_per_sequence = 100
hawkes_process_samples = generate_samples(w, mu, A, num_sequences, max_events_per_sequence)

NameError: name 'A' is not defined